In [8]:
import pandas as pd
import numpy as np
from openai import OpenAI
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import re

In [9]:
openai_api_key = ''

In [10]:
client = OpenAI(api_key=openai_api_key)
articles_df = pd.DataFrame(pd.read_csv('../data/19.csv'))
articles_df

,id,title,category,tags
0,1,한국에서 돈을 가장 빨리 불리는 비법 (feat.13월의 월급 두둑해짐),INVESTMENT,"돈불리기,etf,비법,꿀팁,주식,재테크,연금저축,투자,미국주식"
1,7,대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기!,INVESTMENT,"부동산,재테크,투자"
2,20,애들건강 음식\nㅕ,RELATIONSHIP,"반나나,채소,삶은고기,애호박,요플레"
3,22,운동\n,HOBBY,테그
4,51,[웹소설 전망] 앞으로 웹소설을 혼자 써서 성공하기는 힘들 겁니다,SIDE_JOB,"전망,웹소설,성공비결,공동창작,콘텐츠,성공공식,집단창작"
5,52,웹소설로 돈 벌려면 이 3가지를 익히셔야 합니다,SIDE_JOB,"세컨드잡,웹소설,성공비결,N잡러,N잡,성공공식,인기작품"
6,53,"웹소설, 같이 쓰면 얼마나 더 나아지는지 보여드림",SIDE_JOB,"웹소설,부업,성공비결,N잡,N잡러,성공공식,인기작품"
7,54,웹소설을 같이 쓴다는 것,SIDE_JOB,"1등,웹소설,카카오페이지"
8,55,카카오페이지 웹소설 1위 찍다!,SIDE_JOB,"1등,웹소설,카카오페이지,인기작품"
9,64,발로란트,GAME,"발로란트,세이지,게임,로터스"


In [11]:
articles_df["processed_text"] = articles_df.apply(lambda x: re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "",
                                                                   f"{x['title']} {' '.join(x['tags'].split(','))}"),
                                                  axis=1)


In [12]:
stop_words = pd.read_csv('../data/stop_words_ko.csv')
stop_words


,stop_words
0,아
1,휴
2,아이구
3,아이쿠
4,아이고
...,...
770,원
771,잘
772,통하
773,소리


In [13]:
from nltk.tokenize import word_tokenize

articles_df["processed_text"] = (
    articles_df.apply(lambda x: ' '.join(
        [word for word in word_tokenize(x['processed_text']) if word not in stop_words['stop_words'].tolist()]),
                      axis=1))

articles_df

,id,title,category,tags,processed_text
0,1,한국에서 돈을 가장 빨리 불리는 비법 (feat.13월의 월급 두둑해짐),INVESTMENT,"돈불리기,etf,비법,꿀팁,주식,재테크,연금저축,투자,미국주식",한국에서 돈을 가장 빨리 불리는 비법 feat13월의 월급 두둑해짐 돈불리기 etf...
1,7,대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기!,INVESTMENT,"부동산,재테크,투자",대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기 부동산 재테크 투자
2,20,애들건강 음식\nㅕ,RELATIONSHIP,"반나나,채소,삶은고기,애호박,요플레",애들건강 음식 반나나 채소 삶은고기 애호박 요플레
3,22,운동\n,HOBBY,테그,운동 테그
4,51,[웹소설 전망] 앞으로 웹소설을 혼자 써서 성공하기는 힘들 겁니다,SIDE_JOB,"전망,웹소설,성공비결,공동창작,콘텐츠,성공공식,집단창작",웹소설 전망 앞으로 웹소설을 써서 성공하기는 힘들 겁니다 전망 웹소설 성공비결 공동...
5,52,웹소설로 돈 벌려면 이 3가지를 익히셔야 합니다,SIDE_JOB,"세컨드잡,웹소설,성공비결,N잡러,N잡,성공공식,인기작품",웹소설로 돈 벌려면 3가지를 익히셔야 합니다 세컨드잡 웹소설 성공비결 N잡러 N잡 ...
6,53,"웹소설, 같이 쓰면 얼마나 더 나아지는지 보여드림",SIDE_JOB,"웹소설,부업,성공비결,N잡,N잡러,성공공식,인기작품",웹소설 쓰면 나아지는지 보여드림 웹소설 부업 성공비결 N잡 N잡러 성공공식 인기작품
7,54,웹소설을 같이 쓴다는 것,SIDE_JOB,"1등,웹소설,카카오페이지",웹소설을 쓴다는 1등 웹소설 카카오페이지
8,55,카카오페이지 웹소설 1위 찍다!,SIDE_JOB,"1등,웹소설,카카오페이지,인기작품",카카오페이지 웹소설 1위 찍다 1등 웹소설 카카오페이지 인기작품
9,64,발로란트,GAME,"발로란트,세이지,게임,로터스",발로란트 발로란트 세이지 게임 로터스


In [14]:
def get_embedding(text: str, model="text-embedding-3-large"):
    text = text.replace('\n', ' ')
    return client.embeddings.create(input=[text], model=model).data[0].embedding


articles_df['embedding'] = articles_df.apply(lambda x: get_embedding(x['processed_text']), axis=1)


# articles_df['embedding'] = articles_df.apply(lambda x: get_embedding(f"{x['title']} - {' '.join(x['tags'].split(','))}" ), axis=1)


# with open( "embedded-article-with-tag-content", "rb" ) as file:
#     articles_df = pickle.load(file)


In [35]:
from sklearn.cluster import KMeans

kmeans = KMeans(random_state=0)  
clusters = kmeans.fit_predict(articles_df['embedding'].tolist())
articles_df['cluster'] = clusters


In [49]:
import joblib

joblib.dump(kmeans, "kmeans_model.pkl")

['kmeans_model.pkl']

In [36]:

with open("embedded-article-with-tag-category-large", "wb") as file:
    pickle.dump(articles_df, file)

articles_df

,id,title,category,tags,processed_text,embedding,cluster
0,1,한국에서 돈을 가장 빨리 불리는 비법 (feat.13월의 월급 두둑해짐),INVESTMENT,"돈불리기,etf,비법,꿀팁,주식,재테크,연금저축,투자,미국주식",한국에서 돈을 가장 빨리 불리는 비법 feat13월의 월급 두둑해짐 돈불리기 etf...,"[0.0125399986281991, -0.03982853516936302, -0....",2
1,7,대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기!,INVESTMENT,"부동산,재테크,투자",대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기 부동산 재테크 투자,"[-0.004191316198557615, -0.01968361809849739, ...",2
2,20,애들건강 음식\nㅕ,RELATIONSHIP,"반나나,채소,삶은고기,애호박,요플레",애들건강 음식 반나나 채소 삶은고기 애호박 요플레,"[-0.023416705429553986, -0.017992764711380005,...",5
3,22,운동\n,HOBBY,테그,운동 테그,"[-0.0049162860959768295, -0.020620351657271385...",7
4,51,[웹소설 전망] 앞으로 웹소설을 혼자 써서 성공하기는 힘들 겁니다,SIDE_JOB,"전망,웹소설,성공비결,공동창작,콘텐츠,성공공식,집단창작",웹소설 전망 앞으로 웹소설을 써서 성공하기는 힘들 겁니다 전망 웹소설 성공비결 공동...,"[-0.0260607972741127, -0.042585890740156174, -...",6
5,52,웹소설로 돈 벌려면 이 3가지를 익히셔야 합니다,SIDE_JOB,"세컨드잡,웹소설,성공비결,N잡러,N잡,성공공식,인기작품",웹소설로 돈 벌려면 3가지를 익히셔야 합니다 세컨드잡 웹소설 성공비결 N잡러 N잡 ...,"[0.025320861488580704, -0.0567748062312603, -0...",1
6,53,"웹소설, 같이 쓰면 얼마나 더 나아지는지 보여드림",SIDE_JOB,"웹소설,부업,성공비결,N잡,N잡러,성공공식,인기작품",웹소설 쓰면 나아지는지 보여드림 웹소설 부업 성공비결 N잡 N잡러 성공공식 인기작품,"[0.0012035799445584416, -0.04102521762251854, ...",1
7,54,웹소설을 같이 쓴다는 것,SIDE_JOB,"1등,웹소설,카카오페이지",웹소설을 쓴다는 1등 웹소설 카카오페이지,"[0.004184327088296413, -0.03774350509047508, -...",1
8,55,카카오페이지 웹소설 1위 찍다!,SIDE_JOB,"1등,웹소설,카카오페이지,인기작품",카카오페이지 웹소설 1위 찍다 1등 웹소설 카카오페이지 인기작품,"[0.005606654100120068, -0.0493680015206337, -0...",1
9,64,발로란트,GAME,"발로란트,세이지,게임,로터스",발로란트 발로란트 세이지 게임 로터스,"[-0.02611580863595009, -0.030596265569329262, ...",7


In [37]:
from pinecone import Pinecone
from pinecone import PodSpec

pc = Pinecone(api_key='')

index_name = 'vectordbprac'
dimension = 3072
metric = 'euclidean'
spec = PodSpec('gcp-starter')

if index_name in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.delete_index(index_name)

pc.create_index(index_name, dimension=dimension, metric=metric, spec=spec)

index = pc.Index(index_name)

index_stats = index.describe_index_stats()
print(index_stats)

{'dimension': 3072,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


In [38]:
list_of_dict = articles_df.apply(lambda row: {
    'id': str(row['id']),
    'values': row['embedding'],
    'metadata': {
        'category': row['category'],
        'tags': row['tags'],
        'cluster' : row['cluster'],
        'title' : row['title']
    }}, axis=1).to_list()

list_of_dict

[{'id': '1',
  'values': [0.0125399986281991,
   -0.03982853516936302,
   -0.004317253362387419,
   -0.02179838716983795,
   -0.020850088447332382,
   0.025055043399333954,
   -0.026577310636639595,
   -0.0014871752355247736,
   -0.01548471488058567,
   0.028174445033073425,
   -0.046641308814287186,
   -0.011966029182076454,
   0.007112238090485334,
   -0.05664835497736931,
   0.015272595919668674,
   0.014411640353500843,
   0.040028177201747894,
   -0.0072744470089674,
   0.0021991790272295475,
   -0.017431221902370453,
   -0.0030304996762424707,
   0.004014671314507723,
   0.008497253060340881,
   -0.00181393267121166,
   -0.011516834609210491,
   0.0038524623960256577,
   0.022260058671236038,
   0.003002425190061331,
   -0.02043832652270794,
   -0.013662983663380146,
   0.03621002659201622,
   0.00882790982723236,
   -0.01680734194815159,
   -0.028923101723194122,
   0.02428143098950386,
   0.020113909617066383,
   -0.02188573032617569,
   0.02413170039653778,
   0.03014590777456

In [39]:
index.upsert(
    vectors=list_of_dict
)


{'upserted_count': 31}

In [40]:
query_test_description = word_tokenize(re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", "연애 잘 하고 싶은데"))
query_test_description = ' '.join([word for word in query_test_description if not word in stop_words])

query_embedding = get_embedding(query_test_description)


In [51]:
loaded_model = joblib.load("kmeans_model.pkl")

loaded_model.predict([query_embedding])

array([5], dtype=int32)

In [26]:

index.query(
    vector=query_embedding,
    top_k=3,
    include_values=False,
    include_metadata=True
)

{'matches': [{'id': '114',
              'metadata': {'category': 'RELATIONSHIP',
                           'cluster': 5.0,
                           'tags': '응원해요,연애꿀팁,비법거래소,대처법,비법,가스라이팅,연애비법,연애,극복,연애노하우',
                           'title': '가스라이팅 해대는 애인 참교육할 수 있는 단 1가지 방법'},
              'score': 1.16525531,
              'values': []},
             {'id': '70',
              'metadata': {'category': 'CAREER',
                           'cluster': 6.0,
                           'tags': '일러스트레이터,부업,자기계발,N잡,커리어,수익화,디자이너,자기개발,프리랜서',
                           'title': '조직 밖에서 스스로 돈벌기, 절반의 성공과 절반의 실패'},
              'score': 1.53528845,
              'values': []},
             {'id': '53',
              'metadata': {'category': 'SIDE_JOB',
                           'cluster': 1.0,
                           'tags': '웹소설,부업,성공비결,N잡,N잡러,성공공식,인기작품',
                           'title': '웹소설, 같이 쓰면 얼마나 더 나아지는지 보여드림'},
              'score': 1.54491198,
              'values': []}],
 '

In [34]:
index.query(
     vector=query_embedding,
    top_k=3,
    include_values=False,
    include_metadata=True ,
   filter={"cluster":5}
)

{'matches': [{'id': '114',
              'metadata': {'category': 'RELATIONSHIP',
                           'cluster': 5.0,
                           'tags': '응원해요,연애꿀팁,비법거래소,대처법,비법,가스라이팅,연애비법,연애,극복,연애노하우',
                           'title': '가스라이팅 해대는 애인 참교육할 수 있는 단 1가지 방법'},
              'score': 1.16525531,
              'values': []},
             {'id': '20',
              'metadata': {'category': 'RELATIONSHIP',
                           'cluster': 5.0,
                           'tags': '반나나,채소,삶은고기,애호박,요플레',
                           'title': '애들건강 음식\nㅕ'},
              'score': 1.5689497,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}

In [ ]:
articles_df

In [ ]:
query_test_description = word_tokenize(re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", "부자 되는 법"))
query_test_description = ' '.join([word for word in query_test_description if not word in stop_words])

query_embedding = get_embedding(query_test_description)

articles_df['similarity'] = articles_df['embedding'].apply(
    lambda x: cosine_similarity(np.array(x).reshape(1, -1), np.array(query_embedding).reshape(1, -1))[0][0])
selected_df = articles_df.sort_values(by='similarity', ascending=False)

print("검색어 :" + query_test_description)
selected_df

In [ ]:
def calculate_diversity(df, column_name):
    # 각각의 임베딩끼리 모두 pairwise cosine similarity를 계산
    embeddings = np.vstack(df[column_name])
    cosine_sim = cosine_similarity(embeddings)

    # self-comparisons (diagonal elements)를 제외하고 cosine similarity 계산
    np.fill_diagonal(cosine_sim, np.nan)  # 본인과의 similarity는 제외
    avg_distance = np.nanmean(cosine_sim)

    return cosine_sim, avg_distance


diversity_score = {k: calculate_diversity(articles_df.loc[articles_df['cluster'] == k], 'embedding')[1] for k in
                   range(0, articles_df['cluster'].drop_duplicates().shape[0])}
diversity_score

In [ ]:
cluster = articles_df.loc[articles_df['cluster'] == 2]
cluster

In [ ]:
cluster = articles_df.loc[articles_df['cluster'] == 1]
cluster

In [ ]:
cluster = articles_df.loc[articles_df['cluster'] == 0]
cluster